# Kyeongho Min | KMin@scu.edu | Lab 7

In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
%pylab inline
from sklearn.cluster import KMeans

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv('doglicenses.csv')

In [4]:
df.head()

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Lifetime Neutered Male,MIXED,WHITE/BLACK,BUDDY,15140,2099,5/27/2009 14:44
1,Dog Lifetime Spayed Female,GER SHORTHAIR POINT,SPOTTED,CABELA,15236,2099,6/23/2009 15:31
2,Dog Lifetime Male,BEAGLE MIX,BLACK/BROWN,WATSON,15106,2099,6/15/2011 13:37
3,Dog Lifetime Neutered Male,LABRADOR RETRIEVER,YELLOW,WINSTON,15120,2099,10/18/2019 12:43
4,Dog Senior Lifetime Spayed Female,BORD COLLIE MIX,WHITE/BLACK,SADIE,15227,2099,6/5/2018 12:39


In [5]:
df.shape

(31234, 7)

In [6]:
df.dtypes

LicenseType    object
Breed          object
Color          object
DogName        object
OwnerZip        int64
ExpYear         int64
ValidDate      object
dtype: object

In [7]:
top_10_names=df['DogName'].value_counts().nlargest(10).index.values
top_10_names

array(['BELLA', 'LUCY', 'MOLLY', 'BAILEY', 'DAISY', 'SADIE', 'MAX',
       'CHARLIE', 'BUDDY', 'MAGGIE'], dtype=object)

In [8]:
df['DogName'] = df['DogName'].apply(lambda x: x if x in top_10_names else 'other')

In [9]:
data = df[['LicenseType', 'Breed', 'Color','DogName']]

In [10]:
ml_data = pd.get_dummies(data)

In [11]:
ml_data = ml_data[:1000]
ml_data

,LicenseType_Dog Inter County Transfer - Lifetime,LicenseType_Dog Lifetime Duplicate,LicenseType_Dog Lifetime Female,LicenseType_Dog Lifetime Male,LicenseType_Dog Lifetime Neutered Male,LicenseType_Dog Lifetime Spayed Female,LicenseType_Dog Out of County Transfer - Lifetime,LicenseType_Dog Senior Lifetime Female,LicenseType_Dog Senior Lifetime Male,LicenseType_Dog Senior Lifetime Neutered Male,...,DogName_BELLA,DogName_BUDDY,DogName_CHARLIE,DogName_DAISY,DogName_LUCY,DogName_MAGGIE,DogName_MAX,DogName_MOLLY,DogName_SADIE,DogName_other
0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
996,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
998,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering

bestSil = -1
for k in range(2,6):
    print ('k= %d ' % k)
    clus = [KMeans(n_clusters=k), Birch(n_clusters=k), AgglomerativeClustering(n_clusters=k)]
    #clus = [KMeans(n_clusters=k,n_jobs=-1)]
    for cl in clus:
        res = cl.fit(ml_data)
        sil = metrics.silhouette_score(ml_data, res.labels_)
        print (str(cl)[:6] + ' with k=' +str(k) + ": " + str(round(sil,4)))
        if (sil > bestSil):
            bestSil = sil
            bestCl = cl
            bestK = k
print('***********************************************')
print ('Best algorithm is... ' + str(bestCl)[:6] + '  with k=' +str(bestK) )
print('**********************')
print ('With Silhouette Score ' + str(bestSil))

k= 2 
KMeans with k=2: 0.1107
Birch( with k=2: 0.0599
Agglom with k=2: 0.066
k= 3 
KMeans with k=3: 0.142
Birch( with k=3: 0.0609
Agglom with k=3: 0.0599
k= 4 
KMeans with k=4: 0.1548
Birch( with k=4: 0.0648
Agglom with k=4: 0.0487
k= 5 
KMeans with k=5: 0.1116
Birch( with k=5: 0.0584
Agglom with k=5: 0.0525
***********************************************
Best algorithm is... KMeans  with k=4
**********************
With Silhouette Score 0.154756617328301


In [13]:
for column in df.columns:
    print(column,' : ',df[column].nunique())

LicenseType  :  11
Breed  :  283
Color  :  78
DogName  :  11
OwnerZip  :  136
ExpYear  :  1
ValidDate  :  20825


# Classification: Predicting the dog license type with breed and color of the dog

In [14]:
X = pd.get_dummies(df[['Breed','Color']])

In [39]:
Y = df['LicenseType']

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=42,
)

### Logistic Regression for Multi-class classification

In [49]:
from sklearn.linear_model import LogisticRegression

clf1 = LogisticRegression(solver='newton-cg',multi_class='multinomial',random_state=0).fit(X_train,y_train)
print('Score : ',clf1.score(X_test, y_test))
clf1.predict(X_test)

Score :  0.3986767687546687


array(['Dog Lifetime Spayed Female', 'Dog Lifetime Neutered Male',
       'Dog Lifetime Neutered Male', ..., 'Dog Lifetime Neutered Male',
       'Dog Lifetime Spayed Female', 'Dog Lifetime Spayed Female'],
      dtype=object)

### Decision Tree

In [46]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(X=X_train, y=y_train)
print('Score : ',clf.score(X=X_test, y=y_test)) 
clf.predict(X_test)

Score :  0.3805356952299648


array(['Dog Lifetime Spayed Female', 'Dog Lifetime Neutered Male',
       'Dog Lifetime Spayed Female', ..., 'Dog Lifetime Neutered Male',
       'Dog Lifetime Neutered Male', 'Dog Lifetime Spayed Female'],
      dtype=object)